<a href="https://colab.research.google.com/github/sufiyansayyed19/LLM_Learning/blob/main/W2D2_3_Gradio_Designs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Design:1
can add vanilla css

Default Gradio looks like a government form (gray, boxy, boring).
**Modern Gradio** looks like a SaaS product (Glassmorphism, Shadows, Rounded corners, Custom Fonts).

To achieve this, we need to use three "Pro" techniques:
1.  **`gr.themes`**: Using pre-built styling kits.
2.  **`gr.HTML`**: Injecting custom HTML headers/logos.
3.  **Custom CSS**: The secret weapon to change colors, remove borders, and add animations.

Let's create a **"Master Design Notebook"**. Open a new notebook and name it `Modern_UI_Design.ipynb`.

Here is the code for a **"Next-Gen LLM Studio"** interface.

### 🎨 The "Modern Dark Glass" Design

This code implements:
*   **Glassmorphism**: Semi-transparent backgrounds.
*   **Custom Fonts**: Imports "Inter" from Google Fonts.
*   **Pill Buttons**: Fully rounded, gradient buttons.
*   **Invisible Inputs**: Removing the ugly gray borders from textboxes.

In [2]:
!pip install -q gradio litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 18.9 MB/s eta 0:00:00


In [10]:
import gradio as gr
from litellm import completion
import os
import warnings
from google.colab import userdata

# Silence warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# --- UPDATED CUSTOM CSS (Unchanged) ---
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;800&display=swap');

body, .gradio-container {
    font-family: 'Inter', sans-serif !important;
}

/* HERO HEADER */
.hero-box {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 40px;
    border-radius: 20px;
    text-align: center;
    box-shadow: 0 10px 30px rgba(0,0,0,0.2);
    margin-bottom: 20px;
}
.hero-title {
    font-size: 3em;
    font-weight: 800;
    margin: 0;
    text-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.hero-subtitle {
    font-size: 1.2em;
    opacity: 0.9;
    margin-top: 10px;
    font-weight: 400;
}

/* INPUT BOX */
.modern-input textarea {
    background-color: #ffffff !important;
    color: #1f2937 !important;
    border: 2px solid #e5e7eb !important;
    border-radius: 15px !important;
    padding: 12px !important;
    font-size: 1em;
}
.modern-input > label > span {
    color: #ffffff !important;  /* White label text */
    font-weight: 600;
    font-size: 0.95em;
}

/* RESULT CARD CONTAINER */
.result-card {
    background: #ffffff !important;
    border-radius: 20px;
    padding: 25px;
    border: 1px solid #e5e7eb !important;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    transition: box-shadow 0.2s ease;
}
.result-card:hover {
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.15);
}

/* FIXED OUTPUT: KILL THE GREY BACKGROUND & ENSURE DARK TEXT */
.result-card .markdown,
.result-card .prose {
    background-color: #ffffff !important;  /* Force white bg */
    color: #1f2937 !important;             /* Dark text */
    border: none !important;
    padding: 0 !important;
    margin: 0 !important;
}

/* Target all text elements inside for consistency */
.result-card p,
.result-card h1, .result-card h2, .result-card h3, .result-card h4,
.result-card li, .result-card strong, .result-card em,
.result-card code, .result-card blockquote {
    background-color: transparent !important;
    color: #1f2937 !important;  /* Dark text everywhere */
    border: none !important;
}

.result-card h1, .result-card h2, .result-card h3 {
    color: #4f46e5 !important;  /* Purple accent for headers */
    font-weight: 700;
}

.result-card code {
    background-color: #f3f4f6 !important;
    color: #dc2626 !important;
    padding: 2px 6px;
    border-radius: 4px;
}

/* BUTTON */
.pill-btn {
    border-radius: 50px !important;
    background: linear-gradient(90deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    color: white !important;
    font-weight: 600 !important;
    font-size: 1.1em !important;
    padding: 12px 24px !important;
    margin-top: 10px;
    transition: transform 0.2s ease;
}
.pill-btn:hover {
    transform: scale(1.05);
    box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
}
"""

# --- UPDATED LOGIC: NOW WITH STREAMING ---
def generate_insight(topic):
    try:
        api_key = userdata.get('OPEN_ROUTE_API_KEY') or os.getenv("OPENROUTER_API_KEY")

        if not api_key:
            yield "⚠️ **Error**: Please add `OPEN_ROUTE_API_KEY` to Colab Secrets (or set `OPENROUTER_API_KEY` env var)."
            return

        prompt = f"Give me a futuristic, optimistic insight about: {topic}. Use Markdown with bold headers."

        # Stream the response
        stream = completion(
            model="openrouter/openai/gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            api_key=api_key,
            stream=True  # Enable streaming
        )

        full_content = ""
        for chunk in stream:
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                delta = chunk.choices[0].delta.content
                full_content += delta
                yield full_content  # Yield cumulative for smooth streaming in Gradio

    except Exception as e:
        yield f"❌ **Error**: {str(e)}\n\n*Tip: Check your API key and model availability.*"

# --- UI (Updated: Enable streaming for output) ---
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    gr.HTML("""
    <div class="hero-box">
        <h1 class="hero-title">✨ Zenith AI</h1>
        <p class="hero-subtitle">The Future of Insight Generation</p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🎯 Your Topic")
            inp = gr.Textbox(
                value="Artificial Intelligence in 2030",
                placeholder="Enter a topic...",
                label="Enter Topic",
                elem_classes="modern-input",
                lines=3
            )
            btn = gr.Button("🔮 Generate Insight", elem_classes="pill-btn")

        with gr.Column(scale=2):
            gr.Markdown("### 💡 AI Output")
            with gr.Group(elem_classes="result-card"):
                out = gr.Markdown("Your insight will appear here...", elem_id="streaming-output")

    # Updated click: Gradio auto-handles generator streaming for Markdown
    btn.click(generate_insight, inp, out)

# Launch (share=True for public link if in Colab)
demo.launch(debug=True, share=True)

/tmp/ipython-input-3326593961.py:149: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3326593961.py:149: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d0be4c1bb56fb2f156.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d0be4c1bb56fb2f156.gradio.live


### 🧐 What to Study in this Code

1.  **`custom_css` string:** This is standard Web CSS. If you know web design, you can paste anything here.
2.  **`elem_classes="pill-btn"`**: This is how we link a Python component to a CSS class.
    *   Python: `gr.Button(elem_classes="pill-btn")`
    *   CSS: `.pill-btn { ... }`
3.  **`gr.HTML(...)`**: This allows you to break free from Gradio's layout engine entirely. The colorful header at the top is pure HTML/CSS.
4.  **`gr.Group()`**: This groups multiple items together so you can put a border/shadow around all of them (The "Result Card" effect).

In [23]:
import gradio as gr
from litellm import completion
import os
import warnings
from google.colab import userdata

# Silence warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# --- LOGIC (Streaming enabled) ---
def generate_insight(topic):
    try:
        api_key = userdata.get('OPEN_ROUTE_API_KEY') or os.getenv("OPENROUTER_API_KEY")

        if not api_key:
            yield "⚠️ **Error**: Please add OPEN_ROUTE_API_KEY to Colab Secrets (or set OPENROUTER_API_KEY env var)."
            return

        prompt = f"Give me a futuristic, optimistic insight about: {topic}. Use Markdown with bold headers. Ensure the text is error-free, concise, and engaging—no typos or repetitions."

        # Stream the response
        stream = completion(
            model="openrouter/openai/gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            api_key=api_key,
            stream=True
        )

        full_content = ""
        for chunk in stream:
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                delta = chunk.choices[0].delta.content
                full_content += delta
                yield full_content

    except Exception as e:
        yield f"❌ **Error**: {str(e)}\n\n*Tip: Check your API key and model availability.*"

# --- COMPLETELY REWRITTEN CSS ---
custom_css = """
<style>
    /* FORCE OVERRIDE ALL GRADIO DEFAULTS */
    * {
        box-sizing: border-box;
    }

    /* Main container background */
    .gradio-container,
    body {
        background: linear-gradient(135deg, #1e1b4b 0%, #312e81 100%) !important;
        min-height: 100vh;
    }

    /* CRITICAL: Make ALL textareas visible */
    textarea,
    input,
    .gr-text-input,
    .gr-box textarea,
    textarea[data-testid="textbox"],
    label textarea,
    div[class*="input"] textarea,
    div[class*="textbox"] textarea {
        background-color: #ffffff !important;
        background: #ffffff !important;
        color: #1f2937 !important;
        border: 2px solid #d1d5db !important;
        border-radius: 12px !important;
        padding: 16px !important;
        font-size: 16px !important;
        min-height: 120px !important;
        width: 100% !important;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
    }

    textarea::placeholder {
        color: #9ca3af !important;
    }

    textarea:focus {
        border-color: #667eea !important;
        box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.3) !important;
        outline: none !important;
    }

    /* Hero Section */
    #hero-section {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 48px 32px;
        border-radius: 24px;
        text-align: center;
        margin-bottom: 32px;
        box-shadow: 0 20px 50px rgba(102, 126, 234, 0.3);
    }

    .hero-title {
        font-size: 3.5rem;
        font-weight: 900;
        color: white;
        margin: 0 0 12px 0;
        text-shadow: 0 4px 20px rgba(0, 0, 0, 0.2);
    }

    .hero-subtitle {
        font-size: 1.5rem;
        color: rgba(255, 255, 255, 0.95);
        font-weight: 300;
        margin: 0;
    }

    /* Section Headers */
    .custom-header {
        color: white !important;
        font-size: 1.5rem;
        font-weight: 700;
        margin-bottom: 16px;
        text-shadow: 0 2px 10px rgba(0, 0, 0, 0.3);
    }

    /* Button Styling */
    button[id$="generate-btn"],
    #generate-btn,
    .primary-btn {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        color: white !important;
        font-weight: 700 !important;
        font-size: 1.1rem !important;
        padding: 20px 32px !important;
        border-radius: 9999px !important;
        border: none !important;
        box-shadow: 0 10px 30px rgba(102, 126, 234, 0.4) !important;
        transition: all 0.3s ease !important;
        width: 100%;
        cursor: pointer !important;
    }

    button[id$="generate-btn"]:hover,
    #generate-btn:hover {
        transform: translateY(-2px) scale(1.02) !important;
        box-shadow: 0 15px 40px rgba(102, 126, 234, 0.5) !important;
    }

    /* Output Card Container */
    #output-card,
    .output-container {
        background: white !important;
        border-radius: 24px !important;
        padding: 32px !important;
        box-shadow: 0 20px 50px rgba(0, 0, 0, 0.1) !important;
        border: 1px solid #e5e7eb !important;
        min-height: 200px !important;
    }

    /* CRITICAL: Force ALL markdown content to be visible */
    .markdown,
    .prose,
    .gr-prose,
    #output-card .markdown,
    #output-card p,
    #output-card h1,
    #output-card h2,
    #output-card h3,
    #output-card strong,
    #output-card span,
    #output-card div {
        background: transparent !important;
        color: #1f2937 !important;
    }

    /* Markdown headings */
    .markdown h1,
    .markdown h2,
    .markdown h3,
    #output-card h1,
    #output-card h2,
    #output-card h3 {
        color: #667eea !important;
        font-weight: 700 !important;
        margin-top: 24px !important;
        margin-bottom: 16px !important;
    }

    /* Markdown paragraphs */
    .markdown p,
    #output-card p {
        color: #1f2937 !important;
        font-size: 1.1rem !important;
        line-height: 1.8 !important;
        margin-bottom: 16px !important;
    }

    /* Strong/bold text */
    .markdown strong,
    #output-card strong {
        color: #111827 !important;
        font-weight: 700 !important;
    }

    /* Code blocks */
    .markdown code,
    #output-card code {
        background: #f3f4f6 !important;
        color: #dc2626 !important;
        padding: 4px 8px !important;
        border-radius: 6px !important;
        font-family: monospace !important;
    }

    /* Lists */
    .markdown ul,
    .markdown ol,
    #output-card ul,
    #output-card ol {
        color: #1f2937 !important;
        padding-left: 24px !important;
    }

    .markdown li,
    #output-card li {
        color: #1f2937 !important;
        margin-bottom: 8px !important;
    }

    /* Remove any interfering labels */
    label.svelte-1b6s6s {
        display: none !important;
    }

    /* Responsive */
    @media (max-width: 768px) {
        .hero-title {
            font-size: 2.5rem;
        }
        .hero-subtitle {
            font-size: 1.25rem;
        }
    }
</style>
"""

# --- UI ---
demo = gr.Blocks(css=custom_css, theme=gr.themes.Soft())

with demo:

    # Hero Section
    gr.HTML("""
        <div id="hero-section">
            <h1 class="hero-title">✨ Zenith AI</h1>
            <p class="hero-subtitle">The Future of Insight Generation – Streamed in Real-Time</p>
        </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.HTML('<div class="custom-header">🎯 Your Topic</div>')
            inp = gr.Textbox(
                value="Artificial Intelligence in 2030",
                placeholder="Enter a topic for futuristic insights...",
                lines=4,
                max_lines=6,
                show_label=False
            )
            btn = gr.Button("🔮 Generate Insight", elem_id="generate-btn")

        with gr.Column(scale=2):
            gr.HTML('<div class="custom-header">💡 AI Output</div>')
            with gr.Group(elem_id="output-card"):
                out = gr.Markdown(
                    "Your streamed insight will appear here...",
                    show_label=False
                )

    # Event handler
    btn.click(generate_insight, inputs=inp, outputs=out)

# Launch
demo.launch(debug=True, share=True)

/tmp/ipython-input-1918745101.py:241: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  demo = gr.Blocks(css=custom_css, theme=gr.themes.Soft())
/tmp/ipython-input-1918745101.py:241: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  demo = gr.Blocks(css=custom_css, theme=gr.themes.Soft())


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://52bb8c2469572ee343.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://52bb8c2469572ee343.gradio.live


In [25]:
import gradio as gr
from litellm import completion
import os
import warnings
from google.colab import userdata

# Silence warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# --- LOGIC (Streaming enabled) ---
def generate_insight(topic):
    try:
        api_key = userdata.get('OPEN_ROUTE_API_KEY') or os.getenv("OPENROUTER_API_KEY")

        if not api_key:
            yield "⚠️ **Error**: Please add OPEN_ROUTE_API_KEY to Colab Secrets (or set OPENROUTER_API_KEY env var)."
            return

        prompt = f"Give me a futuristic, optimistic insight about: {topic}. Use Markdown with bold headers. Ensure the text is error-free, concise, and engaging—no typos or repetitions."

        # Stream the response
        stream = completion(
            model="openrouter/openai/gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            api_key=api_key,
            stream=True
        )

        full_content = ""
        for chunk in stream:
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                delta = chunk.choices[0].delta.content
                full_content += delta
                yield full_content

    except Exception as e:
        yield f"❌ **Error**: {str(e)}\n\n*Tip: Check your API key and model availability.*"

# --- TAILWIND + CUSTOM CSS HYBRID ---
custom_html_css = """
<script src="https://cdn.tailwindcss.com"></script>
<script>
    tailwind.config = {
        theme: {
            extend: {
                colors: {
                    'zenith-purple': '#667eea',
                    'zenith-indigo': '#764ba2',
                }
            }
        }
    }
</script>
<style>
    /* Gradio-specific overrides - these MUST come after Tailwind */

    /* Main container */
    .gradio-container {
        background: linear-gradient(135deg, #1e1b4b 0%, #312e81 100%) !important;
        min-height: 100vh !important;
    }

    /* CRITICAL FIX 1: Make input textbox visible */
    textarea,
    input[type="text"],
    .gr-box textarea,
    textarea[data-testid="textbox"] {
        background-color: white !important;
        color: #1f2937 !important;
        border: 2px solid #d1d5db !important;
        border-radius: 12px !important;
        padding: 16px !important;
        font-size: 16px !important;
        min-height: 120px !important;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
    }

    textarea:focus,
    input[type="text"]:focus {
        border-color: #667eea !important;
        box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.3) !important;
        outline: none !important;
    }

    textarea::placeholder {
        color: #9ca3af !important;
    }

    /* CRITICAL FIX 2: Make output text visible */
    .markdown,
    .prose,
    #output-card,
    #output-card * {
        background: transparent !important;
    }

    .markdown,
    .markdown p,
    .markdown span,
    .markdown div,
    #output-card,
    #output-card p,
    #output-card span,
    #output-card div {
        color: #1f2937 !important;
    }

    .markdown h1,
    .markdown h2,
    .markdown h3,
    #output-card h1,
    #output-card h2,
    #output-card h3 {
        color: #667eea !important;
        font-weight: 700 !important;
    }

    .markdown strong,
    #output-card strong {
        color: #111827 !important;
        font-weight: 700 !important;
    }

    /* Hide default Gradio labels */
    label.svelte-1b6s6s {
        display: none !important;
    }
</style>
"""

# --- UI USING TAILWIND CLASSES ---
demo = gr.Blocks(css="", theme=gr.themes.Soft())

with demo:

    # Inject Tailwind + Custom CSS
    gr.HTML(custom_html_css)

    # Hero Section - Using Tailwind classes
    gr.HTML("""
        <div class="bg-gradient-to-br from-zenith-purple to-zenith-indigo text-white p-12 rounded-3xl text-center mb-8 shadow-2xl">
            <h1 class="text-6xl font-black mb-3 drop-shadow-lg">✨ Zenith AI</h1>
            <p class="text-2xl opacity-95 font-light">The Future of Insight Generation – Streamed in Real-Time</p>
        </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            # Custom header using Tailwind
            gr.HTML('<div class="text-white text-2xl font-bold mb-4 drop-shadow-lg">🎯 Your Topic</div>')

            inp = gr.Textbox(
                value="Artificial Intelligence in 2030",
                placeholder="Enter a topic for futuristic insights...",
                lines=4,
                show_label=False
            )

            # Button with Tailwind classes
            gr.HTML('<div id="btn-container" class="mt-4"></div>')
            btn = gr.Button(
                "🔮 Generate Insight",
                elem_id="generate-btn"
            )

        with gr.Column(scale=2):
            # Custom header using Tailwind
            gr.HTML('<div class="text-white text-2xl font-bold mb-4 drop-shadow-lg">💡 AI Output</div>')

            # Output card - using elem_classes for proper wrapper
            with gr.Group(elem_id="output-wrapper"):
                out = gr.Markdown(
                    "Your streamed insight will appear here...",
                    elem_id="output-card",
                    show_label=False
                )

    # Event handler
    btn.click(generate_insight, inputs=inp, outputs=out)

# Additional CSS for button and output styling
demo.css = """
    /* Button styling */
    #generate-btn {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        color: white !important;
        font-weight: 700 !important;
        font-size: 1.1rem !important;
        padding: 20px 32px !important;
        border-radius: 9999px !important;
        border: none !important;
        box-shadow: 0 10px 30px rgba(102, 126, 234, 0.4) !important;
        width: 100% !important;
        cursor: pointer !important;
        transition: all 0.3s ease !important;
    }

    #generate-btn:hover {
        transform: translateY(-2px) scale(1.02) !important;
        box-shadow: 0 15px 40px rgba(102, 126, 234, 0.5) !important;
    }

    /* Output wrapper - white card */
    #output-wrapper {
        background: white !important;
        border-radius: 24px !important;
        padding: 32px !important;
        box-shadow: 0 20px 50px rgba(0, 0, 0, 0.1) !important;
        border: 1px solid #e5e7eb !important;
        min-height: 300px !important;
    }

    /* Output card content */
    #output-card {
        background: white !important;
    }

    /* Force all markdown text to be dark and visible */
    #output-wrapper .markdown,
    #output-wrapper .prose,
    #output-card,
    #output-card .markdown,
    #output-card .prose {
        background: white !important;
    }

    #output-wrapper p,
    #output-wrapper span,
    #output-wrapper div,
    #output-card p,
    #output-card span,
    #output-card div,
    #output-card .markdown p,
    #output-card .markdown span,
    #output-card .markdown div {
        color: #1f2937 !important;
        background: transparent !important;
    }

    #output-wrapper h1,
    #output-wrapper h2,
    #output-wrapper h3,
    #output-card h1,
    #output-card h2,
    #output-card h3 {
        color: #667eea !important;
        font-weight: 700 !important;
        background: transparent !important;
    }

    #output-wrapper strong,
    #output-card strong {
        color: #111827 !important;
        font-weight: 700 !important;
    }
"""

# Launch
demo.launch(debug=True, share=True)

/tmp/ipython-input-2301541971.py:132: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  demo = gr.Blocks(css="", theme=gr.themes.Soft())
/tmp/ipython-input-2301541971.py:132: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  demo = gr.Blocks(css="", theme=gr.themes.Soft())


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://061f4c7db5131552b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://061f4c7db5131552b8.gradio.live
